In [3]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 789kB/s eta 0:00:01
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-macosx_10_7_x86_64.whl size=79894 sha256=4082648930725e8f58735f73d82f8e667e87ea5cc4f2306ecc454a2ed3bee8b0
  Stored in directory: /Users/louiszhao/Library/Caches/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install cleanco

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from cleanco import cleanco
import numpy as np
import pandas as pd
from itertools import compress

In [4]:
orig= pd.read_csv("Financial_Data_WIP.csv")
m_a = pd.read_csv("mergers.csv")


In [40]:
m_a

,MA_Date,Firm1,Firm2
0,1/31/71,SCHERING CORP,PLOUGH INC
1,9/22/80,FRED MEYER INC,KOHLBERG KRAVIS ROBERTS & CO
2,11/1/80,RICHARDSON-MERRELL INC,SHAREHOLDERS
3,12/8/80,NEW ENGLAND NUCLEAR CORP,DUPONT
4,1/26/81,EX-LAX PHARMCTCL(CULBRO CORP),SANDOZ AG
...,...,...,...
11976,5/4/20,ORBIS BIOSCIENCES INC,ADARE PHARMACEUTICALS INC
11977,5/4/20,TRANSGENE SA-DUCKCELT CELL LIN,VAXXEL SAS
11978,5/7/20,ELSALYS BIOTECH SAS,MEDIOLANUM FARMACEUTICI SPA
11979,5/11/20,GALLIA FARMACEUTICA LTDA,GADOR SA


In [5]:
orig.loc[orig['Company'] == 'lumara health inc']

,Year,Company,tic,cusip,IPO date,Year Founded,Source,Type,Capital Expenditures,R&D Expense,Earnings,Cash,Tangible Assets,Total Debt,Total Revenue,Total Assets,status,cik,exchange,Company_ID


In [3]:
m_a["Firm1"] = m_a['Firm1'].str.lower()
m_a["Firm2"] = m_a['Firm2'].str.lower()

In [4]:
temp_list = []
it = 0
while(it < 200):
    a= m_a.iloc[it,1]
    b = m_a.iloc[it,2]
    a_match = process.extract(a, orig_list, scorer=fuzz.token_sort_ratio)
    b_match = process.extract(b, orig_list, scorer=fuzz.token_sort_ratio)
    if (a_match[0][1] > 70) & (b_match[0][1]> 70):
        print(a_match[0][1], b_match[0][1] )
        c = a_match[0][0] + "---"+  b_match[0][0]
        print(a, "    ", b)
        print(c)
        temp_list.append(c)
    it = it+1
    
    

NameError: name 'orig_list' is not defined

[]

In [5]:
m_a_companies= m_a["Firm1"].append(m_a["Firm2"])
m_a_companies = m_a_companies.values.tolist()
m_a_companies = np.unique(m_a_companies)
m_a_companies = m_a_companies.tolist()

In [6]:
m_a_companies = [x.lower() for x in m_a_companies]
#m_a_companies = [cleanco(x).clean_name() for x in m_a_companies]


In [53]:
orig_list = np.unique(orig["Company"]).tolist()
#orig_list = [cleanco(x).clean_name() for x in orig_list]
#orig_list = [x.replace("-redh", "") for x in orig_list]
#orig_list = [x.replace("-adr", "") for x in orig_list]
orig_list = [x.lower() for x in orig_list]
orig_list = [x.replace("-old", " -old") for x in orig_list]
orig_list

['14605 inc',
 '1pm industries inc',
 '22nd century group inc',
 '3-dimensional pharmaceutical',
 'astellas institute for regenerative medicine',
 'betterlife pharma inc',
 'chiesi usa inc',
 'cyclo therapeutics inc',
 'horizon pharmaceutical llc',
 'lumara health inc',
 'medicis pharmaceutical  corp',
 'opko biologics',
 'seneca biopharma inc',
 'the natures bounty co',
 'westaim holdings ltd',
 'abbott laboratories',
 'abbvie inc',
 'abeona therapeutics inc',
 'abgenix inc',
 'able laboratories inc',
 'abpro corp -redh',
 'abraxis bioscience inc',
 'ac immune sa',
 'acacia diversified hldgs inc',
 'acadia pharmaceuticals inc',
 'acambis plc',
 'accelerate diagnostics inc',
 'accelerated pharma inc -redh',
 'acceleron pharma inc',
 'accentia biopharmaceuticals',
 'acelity holdings inc -redh',
 'acelrx pharmaceuticals inc',
 'acer therapeutics inc',
 'achaogen inc',
 'achieve life sciences inc',
 'achillion pharmaceuticals',
 'aclaris therapeutics inc',
 'acorda therapeutics inc',
 'ac

In [54]:

m_a_companies_string = " ".join(m_a_companies)
orig_string = " ".join(orig_list)
orig_string = orig_string +" " +  m_a_companies_string
orig_string = orig_string.split()

print(len(m_a_companies_string))
def wordcount(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux
word_dict = wordcount(orig_string)

word_list = sortFreqDict(word_dict)


397749


In [9]:
def sequence_uniqueness(seq):
    return sum(1/word_dict[t]**0.5 for t in seq)

def name_similarity(a, b, limit):
    a_tokens = set(a.split())
    b_tokens = set(b.split())
    a_uniq = sequence_uniqueness(a_tokens)
    b_uniq = sequence_uniqueness(b_tokens)
    
    similarity = sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
    if similarity > limit:
        combined = [a,b,similarity]
        return combined
name_similarity("forest laboratories inc", "forest laboratories -cl a", .5)

['forest laboratories inc', 'forest laboratories -cl a', 0.6978459801917798]

In [100]:
it = 0
col_names = ['m&aCo', 'origCo', 'Ratio']
df_master_A = pd.DataFrame( columns = col_names)
df_master_B = pd.DataFrame( columns = col_names)
df_master = []
while(it < len(m_a)):
    a= m_a.iloc[it,1]
    b = m_a.iloc[it,2]
    c = [name_similarity(a,val, .60) for val in orig_list if name_similarity(a,val, .60) is not None]
    d = [name_similarity(b,val, .60) for val in orig_list if name_similarity(b,val, .60) is not None]
    if len(c) != 0 and len(d) != 0:
        c = find_max(c)
        df_master_A = df_master_A.append(pd.Series(c, index = df_master_A.columns), ignore_index = True)
        d = find_max(d)
        df_master_B = df_master_B.append(pd.Series(d, index = df_master_B.columns), ignore_index = True)
    it = it+1
df_master = pd.concat([df_master_A, df_master_B], axis =1) 


,m&aCo,origCo,Ratio,m&aCo,origCo,Ratio
0,integrated genetics inc,integrated biopharma inc,0.688885,genzyme corp,genzyme corp,1.000000
1,squibb corp,bristol-myers squibb co,0.702032,bristol-myers co,bristol-myers squibb co,0.673639
2,quidel corp,quidel corp,1.000000,quidel corp,quidel corp,1.000000
3,immucor gmbh,immucor inc,0.948370,immucor inc,immucor inc,1.000000
4,meridian diagnostics europe,meridian bioscience inc,0.711386,meridian diagnostics inc,meridian bioscience inc,0.820291
...,...,...,...,...,...,...
321,aevi genomic medicine inc,aevi genomic medicine inc,1.000000,cerecor inc,cerecor inc,1.000000
322,synthorx inc,synthorx inc,1.000000,sanofi sa,sanofi,0.906086
323,amgen astellas biopharma,amgen inc,0.687344,amgen inc,amgen inc,1.000000
324,icagen inc-assets,icagen inc,0.796198,ligand pharmaceuticals inc,ligand pharmaceutical inc,0.902636


In [97]:
aa = "avant immunotherapeutics inc"
test = [name_similarity(aa,val, .60) for val in orig_list if name_similarity(aa,val, .60) is not None]
def find_max(results):
    tempvec = []
    for i in range(len(results)):
        tempvec.append(results[i][2])
    val = max(tempvec)
    index = tempvec.index(val)
    return results[index]
find_max(test)

['avant immunotherapeutics inc',
 'avant immunotherapeutics -old',
 0.8960214548188572]

In [104]:
def print_full(x):
    
    print(x)
    pd.reset_option('display.max_rows')
def highlight_cols(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #select all values to default value - red color'
    #overwrite values grey color
    df.loc[:,:] = 'color: = black'
    df[['M&A_Company1', 'M&A_Company2']] = 'color: blue'
    df[['Original_Company1', 'Original_Company2']] = 'color: red'
    #return color df
    return df    


df_master.columns = ['M&A_Company1', 'Original_Company1', 'Company1_Ratio'
                    ,'M&A_Company2', 'Original_Company2', 'Company2_Ratio']


In [106]:
pd.set_option('display.max_rows', len(df_master))
df_master

,M&A_Company1,Original_Company1,Company1_Ratio,M&A_Company2,Original_Company2,Company2_Ratio
0,integrated genetics inc,integrated biopharma inc,0.688885,genzyme corp,genzyme corp,1.000000
1,squibb corp,bristol-myers squibb co,0.702032,bristol-myers co,bristol-myers squibb co,0.673639
2,quidel corp,quidel corp,1.000000,quidel corp,quidel corp,1.000000
3,immucor gmbh,immucor inc,0.948370,immucor inc,immucor inc,1.000000
4,meridian diagnostics europe,meridian bioscience inc,0.711386,meridian diagnostics inc,meridian bioscience inc,0.820291
5,diagnostic products uk ltd,diagnostic products corp,0.726393,diagnostic products corp,diagnostic products corp,1.000000
6,bone health inc,bone care international inc,0.759169,draxis health inc,draxis health inc,1.000000
7,boston life sciences inc,boston therapeutics inc,0.740378,boston life sciences inc,boston therapeutics inc,0.740378
8,pharmacia ab,pharmacia corp,0.810291,upjohn co,pharmacia & upjohn inc,0.704471
9,dura delivery systems inc,dura pharmaceuticals inc,0.761248,dura pharmaceuticals inc,dura pharmaceuticals inc,1.000000


In [46]:
df_master.columns = ['M&A_Company1', 'Original_Company1', 'Company1_Ratio'
                    ,'M&A_Company2', 'Original_Company2', 'Company2_Ratio']
boolarray = ((df_master['Company1_Ratio']  ==1) &(df_master['Company2_Ratio'] == 1))
boolarray = [not i for i in boolarray]
df_master_filtered = df_master[boolarray]
df_master_filtered

,M&A_Company1,Original_Company1,Company1_Ratio,M&A_Company2,Original_Company2,Company2_Ratio
0,integrated genetics inc,integrated biopharma inc,0.688892,genzyme corp,genzyme corp,1.000000
1,squibb corp,bristol-myers squibb co,0.702001,bristol-myers co,bristol-myers squibb co,0.673639
3,immucor gmbh,immucor inc,0.948362,immucor inc,immucor inc,1.000000
4,meridian diagnostics europe,meridian bioscience inc,0.711380,meridian diagnostics inc,meridian bioscience inc,0.820294
5,diagnostic products uk ltd,diagnostic products corp,0.726360,diagnostic products corp,diagnostic products corp,1.000000
...,...,...,...,...,...,...
316,cerecor inc-pediatric,cerecor inc,0.662456,aytu bioscience inc,aytu bioscience inc,1.000000
317,achillion pharmaceuticals inc,achillion pharmaceuticals,0.990670,alexion pharmaceuticals inc,alexion pharmaceuticals inc,1.000000
320,synthorx inc,synthorx inc,1.000000,sanofi sa,sanofi,0.906086
321,amgen astellas biopharma,amgen inc,0.687335,amgen inc,amgen inc,1.000000


In [107]:
df_master.to_csv('merged_companies.csv')

In [29]:
df_master.columns

Index(['m&aCo', 'origCo', 'Ratio', 'm&aCo', 'origCo', 'Ratio'], dtype='object')